In [ ]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from matplotlib import colormaps
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib.collections import EllipseCollection
from sklearn.preprocessing import scale
sns.set_style('darkgrid')

%matplotlib inline

In [ ]:
def plot_corr_ellipses(data, ax=None, **kwargs):

    M = np.array(data)
    if not M.ndim == 2:
        raise ValueError('data must be a 2D array')
    if ax is None:
        fig, ax = plt.subplots(1, 1, subplot_kw={'aspect':'equal'})
        ax.set_xlim(-0.5, M.shape[1] - 0.5)
        ax.set_ylim(-0.5, M.shape[0] - 0.5)

    # xy locations of each ellipse center
    xy = np.indices(M.shape)[::-1].reshape(2, -1).T

    # set the relative sizes of the major/minor axes according to the strength of
    # the positive/negative correlation
    w = np.ones_like(M).ravel()
    h = 1 - np.abs(M).ravel()
    a = 45 * np.sign(M).ravel()

    ec = EllipseCollection(widths=w, heights=h, angles=a, units='x', offsets=xy,
                           transOffset=ax.transData, array=M.ravel(), **kwargs)
    ax.add_collection(ec)

    # if data is a DataFrame, use the row/column names as tick labels
    if isinstance(data, pd.DataFrame):
        ax.set_xticks(np.arange(M.shape[1]))
        ax.set_xticklabels(data.columns, rotation=90)
        ax.set_yticks(np.arange(M.shape[0]))
        ax.set_yticklabels(data.index)

    return ec


Dans ce projet, nous allons nous intérésser au chargement des stations de vélibs de la ville de Paris. Nous allons pour ce faire considérer les jeux de données 'velibLoading.csv' ainsi que 'velibCoord.csv'. Le premier data frame contient les chargements des différentes sations de vélibs heure par heure au cours d'une semaine. Ce data frame a donc pour variable (=colonne) les Chargements des stations à une heure précise d'un jour de la semaine et pour individus (=lignes), les différentes stations de la ville de Paris que nous considérons.Le chargement des stations est défini comme le quotient/rapport du nombre de vélib disponible dans la station par le nombre de vélib total. On comprend donc que si le chargement vaut 1, la station est rempli de vélib, tandis que, si le chargement vaut 0, la station est vide car ils sont tous actuellement en cours d'utilisation.
Le data frame 'velibCoord.csv' quant à lui contient la longitude de la station, la latitude de la station, le Bonus et le nom de la station en question.Les individus sont les stations de la ville de Paris.
On considèrera 1189 individus, c'est à dire, 1189 stations de vélib.

In [ ]:
#Chargement du jeu de donnée loading
loading = pd.read_csv('velibLoading.csv', delimiter=' ')
print(loading.head())



Le data frame loading est de taille (1189,168). Comme expliqué précedemment, les 1189 individus correspondent aux stations de vélib. Les 168 colonnes viennent des tranches horaires que l'on a utilisé pour décomposer la semaine en tranche horaire de 1heure. On a 24h dans une journée et 7 jours dans la semaine, et si l'on fait 24x7 on obtient 168h donc on a 168 colonnes.

In [ ]:
loading.mean(axis=1).describe()

La commande que nous venons d'utiliser nous donne des informations intéréssantes qui nous seront très utiles par la suite. Tout d'abord, nous voyons qu'il y a 1189 stations de vélibs dans la ville de Paris et que la moyenne en chargement de vélib au cours de la semaine dans la ville de Paris est de 0.38 ( 38%) qui est inférieur à 0.5. Cela signifie que les vélibs des stations sont en moyenne chargé en vélib à 38%, donc en moyenne, 72% des vélibs sont utilisés. Cela nous montre que ce moyen de transport est tout de même très utilisé dans la ville de Paris.
L'écart type nous permet de constater que le chargement en vélib des stations varie d'environ 0.212 par rapport à la moyenne, signfiant que les stations ont un chargement en vélib assez proche de la moyenne en chargement de vélib de la ville de Paris. 
Le chargement minimal de 0.016 nous indique qu'au cours de la semaine certaines stations peuvent avoir une disponibilité très faible en vélib allant jusqu'à  1.6%, signifiant qu'environ 98% des vélibs sont en cours d'utilisation. On peut imaginer que ce cas de figure se produit dans certaines stations aux heures où les parisiens doivent se rendre sur leur lieux de travail.
Le chargement maximal de 0.919 nous montre qu'au contraire certaines stations peuvent avoir un remplissage assez conséquent en vélib au cours de la semaine. Ce qui est intéréssant à noter est que le maximum est différent de 1. Cela nous montrer qu'au cours de la semaine, il n'y a aucune station qui est chargée à 100% en vélib.
Le premier quartile nous indique que 25% des stations ont un chargement inférieur à 0.20. La médiane nous informe que la moitié des stations de Paris ont un chargement moyen inférieur à 0.35 < 0.5. Le troisième quartile quant à lui nous permet d'observer que 75% des stations de vélibs de Paris ont un chargement inférieur à 0.53.
Cela nous montre que ce moyen de transport est tout de même très utilisé dans la ville de Paris.
Dans la suite nous nous pencherons sur un recodage des variables pour l'étude de l'altitude des stations.

In [ ]:
# Chargement du jeu de donnée coord
coord = pd.read_csv('velibCoord.csv',delimiter=' ')
coord.head()
#print(coord.shape)

Le data frame coord est de taille (1189,4).  Comme expliqué précédemment, les 1189 individus correspondent aux stations de vélibs. Dans ce data frame, la variable names nous informe du nom des stations de vélibs des individus, nous retrouvons également la longitude et latitude nous permettant de connaitre la position des différentes stations.

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(coord, lat='latitude', lon='longitude',
                        mapbox_style='open-street-map',
                        zoom=10, opacity=.9,
                        title='Stations de vélibs à Paris',
                        color_discrete_sequence=['black'])

fig.show()

In [ ]:
# Définir les intervalles et les étiquettes pour chaque catégorie
intervals = [0, 0.204574, 0.353352, 0.532460, 0.7, 1.1]
labels = ['très peu chargé', 'peu chargé', 'moyen', 'chargé', 'très chargé']

# Appliquer la transformation
categorical_data = pd.DataFrame()
for column in loading.columns:
    categorical_data[column] = pd.cut(loading[column], bins=intervals, labels=labels, right=False)

# Afficher les premières lignes du nouveau DataFrame
categorical_data.head()

Dans ce projet, nous allons tout d'abord réaliser une analyse descriptive de nos données. Ensuite, nous ferons une une analyse en composante principal(ACP) au vu de la grande dimension de nos données. Par la suite, nous nous pencherons sur les méthodes de clustering tel que K-means, GMM. L'objectif est de détecter des "clusters" afin de pouvoir regrouper les utilisateurs de vélibs selon l'utilisation qu'ils ont. Ces clusters nous permettrons ensuite de prédire les chargements.


# Etude rapide du jeu de donnée( Vérification des données)

Nous allons procéder dans cette partie à une vérification de la validité de nos jeux de données loading et coord. 
Dans un premier temps, nous vérifions que toutes les cases de nos jeu de données sont remplis. 

In [ ]:
#Verification présence de case non remplis dans les 2 jeux de données
print(loading.any().isna().sum().sum())
#POur chacune des colonnes,on verifie si il y a la présnce d'une case non rempli (NA), après on somme les cases NA de cette colonne
#Ensuite on somme les sommes des colonnes. Le résultat retourné pour le jeux de donnée Loading est 0.
#Il n'a donc pas de donnée manquante dans Loading
print(coord.any().isna().sum().sum())
# De même pour Coord.

On vérifie donc que pour chacune des colonnes s'il y a la présence d'une case non rempli (NA), et l'on somme ces potentiels case non remplis de cette colonne, puis on sommes les sommes obtenus pour chaque colonne. Le résultat retourné est 0 pour nos deux jeux de données nous indiquant bien que nos jeux de données ne présentent pas de valeurs manquantes.

In [ ]:
#Verification ligne dupliquées dans les jeux de données
print(loading.duplicated().sum())
#Ici, on va parcourir les lignes et vérifié s'il y a des lignes dupliquées. Ici ce n'est pas le cas.
print(coord.duplicated().sum())

Dans cette cellule, nous effectuons une verification de potentielles lignes dupliquées de nos jeux de données. Ici nous retournons 0 pour nos deux jeux de données nous confirmant la non présence de lignes dupliquées.

In [ ]:
# Etude nom des stations
#print(coord['names'])
coord['names'].duplicated().sum()
#Cela retourne 28
duplicated_names = coord[coord['names'].duplicated()]

# Afficher les noms des stations identiques
print("Noms des stations identiques :")
print(duplicated_names['names'].unique())
#print(duplicated_names)


A présent, nous allons nous pencher sur le nom des stations de vélibs de notre jeu de données coord. Il y'a donc 28 noms de stations qui se répètent, cela s'explique par le fait que certaines stations portent le même nom,. Nous décidons d'afficher le nom de ces stations.


Nous avons donc bien compris et vérifié la véracité ainsi que la complétude de nos données. Nous allons mainteant nous pencher sur l'analyse descriptive de nos données

# Etude du remplissage des stations selon l'heure ou le jour

A présent nous allons nous pencher sur le remplissage des stations selon l'heure de la journée ainsi que selon le jour de la semaine.

In [ ]:
#Evolution de la disponibilité à la station Quai de la RAPEE au cours de la semaine.
i = 4
loading_data = loading.to_numpy()
print(coord)
n_steps = len(loading_data[i])   # number of observed time steps
time    = 24*7  # observed time range

plt.figure(figsize = (20, 6))

plt.plot(loading_data[i])
for j in range(1,7*24+1):
    if j%24==0:
        plt.vlines(x=j,colors='r',ymin=0,ymax=1)
plt.xlabel('Heure')
plt.ylabel('Ratio du nombre de vélib disponible')
plt.title(coord['names'][i+1])

Ce graphique nous montre le remplissage de la station de vélib Quai de la RAPEE au cours de la semaine. Nous avons pris soin de séparer les jours de la semaine à l'aide de lignes rouges.
On observe que le chargement de la station est périodique au cours de la semaine et que la tendance est différente à la fin de la semaine.

In [ ]:
#Evolution de la disponibilité pour 16 stations au cours de la semaine.
#plt.figure(figsize = (1000, 70))
a=3; b=3;
time=[24*(1+j) for j in range(7)]
fig, ax = plt.subplots(a, b,figsize=(30,20), layout='constrained')
fig.suptitle("Affichage de l'évolution de Ratio disponibilité des vélibs pour 16 stations")
for i in range(a):
    for j in range(b):
        k= np.random.choice(np.array([l for l in range(len(loading_data[:,1]))]))
        ax[i,j].plot(loading_data[k,:])
        ax[i,j].set_title(coord['names'][k])
        ax[i,j].vlines(time,0,1,color="black")
for ax in ax.flat:
    ax.set(xlabel='Heure', ylabel='Ratio')

Nous remaquons que les chargements des stations dépendent certes des jours de la semaine( en semaine les chargement sont différents de ceux du week end) mais que cela dépend bien de la station en elle même. Si on prend 2 stations différentes, on aura 2 dynamiques de chargement différentes.

In [ ]:
plt.figure(figsize = (20,9))
n_steps    = loading.shape[1]  # number of observed time steps
time_range = np.linspace(1, n_steps, n_steps)  # observed time range
time_tick  = np.linspace(1, n_steps, 8) 


bp = plt.boxplot(loading_data, widths = 0.75, patch_artist = True)

for box in bp['boxes']:
    box.set_alpha(0.8)
    
for median in bp['medians']:
    median.set(color = "Purple", linewidth=5)
    

    
plt.vlines(x = time_tick, ymin = 0, ymax = 1, 
           colors = "Orange", linestyle = "dotted", linewidth = 5)



plt.xlabel('Heures', fontsize = 20)
plt.ylabel('Chargement en vélibs', fontsize = 20)
plt.title("Boxplots des chargements horaires au cours de la semaine", fontsize = 25)
plt.xticks(ticks = np.arange(0, 168, 5), labels=np.arange(0, 168, 5), fontsize = 15)
plt.yticks(fontsize = 15)

plt.tight_layout()
plt.show()

Ces boxplots représentent le chargement moyen des stations au cours de la semaine. Tout comme pour le graphique précedent, nous séparons les jours de la semaine à l'aide des lignes oranges pour des questions de lisibilité et de simplicité d'interprétation. La courbe violette représente la médiane, l'allure de cette courbe nous permet de comprendre le chargement jouralier et donc hebdomadaire du chargement global des stations de la ville de Paris. Nous pouvons observer un maximum de disponibilité vers 08h et un minimum de disponibilité vers 19h pour les différents jours de la semaine. Le week end le maximum est un peu plus tard dans la journée. 
Nous savons que pour les boxplots, il y'a une ligne pour le 1er quartile , une seconde pour le 2nd quartile ( médiane) et une troisième pour le troisième quartile. Nous observons une disymétrie des données par rapport à la médiane. En effet, si l'on remaque que la différence entre le troisième quartile et la médiane est plus grande que la différence entre le premier quartile est la médiane reflettant bien la disymétrie des données. Cela confirme les résultats obtenu au début à l'aide de la commande python describe. En effet, on avait vu que :
la moyenne était de 0.381622, le premier quartile était de 0.204574, la médiane était de 0.353352 et le troisième quartile était de 0.532460. Et la différence médiane premier quartile est d'environ 0.15, ce qui est inférieure à la différence entre le troisième quartile et la médiane étant d'environ 0.18 et 0.15<0.18.
D'autre part la dispersion des données à l'air constante en semaine car les courbes on à peu prés la même allure. Cependant, on observe que le samedi, le ratio de disponibilité des données dépasse 0.8. Et que le dimanche l'allure de la disponibilité est proche du samedi mais avec des valeurs de chargement plus proche de celle des jours de la semaine.

In [ ]:
mean_per_hour_per_day = loading.mean(axis = 0).to_numpy()
mean_per_hour_per_day = mean_per_hour_per_day.reshape((7, 24))

mean_per_hour = mean_per_hour_per_day.mean(axis=0)



days = ["Monday", "Tuesday", "Wednesday","Thursday", "Friday", "Saturday", "Sunday"]
plt.figure(figsize = (15,10))

plt.plot(mean_per_hour_per_day.transpose())
plt.plot(mean_per_hour, color = "black", linewidth = 3)

plt.xlabel('Hourly loading, averaged over all stations', fontsize = 20)
plt.ylabel('Loading', fontsize = 20)
plt.legend(days + ['Weekly'])
plt.xticks(ticks = np.arange(0,24,4), labels=np.arange(0,24,4), fontsize = 15)
  
plt.tight_layout
plt.show()

On affiche le chargement moyen de chaque jour de la semaine, on observe que les jours de la semaine suivent la même tendance qui est assez différente par rapport à celle des jours de weekend.
On observe 2 pics de chargement faible à 9h et à 19h, cela corred=spond aux heures où les gens partent au travail et rentrent du travail (heures du transit)

In [ ]:
print(loading.mean())
plt.figure(0)
plt.plot(loading.mean(axis=0))
plt.axhline(y = np.mean(loading.mean(axis=0)), color = 'r')
plt.title("Average loading by stations")
#moyenne sur les stations
plt.figure(1)
plt.plot(loading.mean(axis=1))
plt.axhline(y = np.mean(loading.mean(axis=1)), color = 'r')
plt.title("Average hourly loading")
#moyenne sur le temps

Le second graphe représente le chargement moyen en vélibs pour les stations au cours de la semaine, chacune des barres nous indique la moyenne en chargement de vélib de la station au cours de la semaine. Ce graphe nous confirme ce que l'on voyait précédemment puisque l'on observe que certaines stations ont un chargement moyen très fort indiquant que leurs vélibs ne sont pas beaucoup utilisés. Tandis que d'autres stations ont un chargement moyen assez faible montrant que ce sont des stations qui voient leurs vélibs être fortement utilisés par les parisiens. De plus, nous voyons que la ligne rouge correspondant à la  moyenne en chargement de la moyenne de chargement des stations est en dessous de 0.4 nous indiquant que les stations ont géneralement plus de places libres que de places occupées.

In [ ]:

## Simple 2D representation
# Loading at 6pm, depending on the day of the week

h = 18
# h =0
#h=12
hours = np.arange(h, 168, 24)

load_per_hour = loading_data[:, hours]

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# --- #

s, m = 10, 3
k = 1 + len(days)//m

fig = plt.figure(figsize=(s+1, s))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=.3, wspace=.25)

for (i,d) in enumerate(days):
    ax = fig.add_subplot(k, m, i+1)
    im = ax.scatter(coord.longitude, coord.latitude, c = load_per_hour[:,i])
    plt.colorbar(im)
    
    ax.set_title('Stations loading - ' + d + ' {} h'.format(h))
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.tick_params(axis='x')
    ax.tick_params(axis='y')


plt.show()


Ces graphiques nous montrent le remplissage moyen des différentes stations de Paris à une heure précise de la semaine. La latitude et la longitude nous permette de savoir ou sont situées ces stations dans la ville de Paris. Ce qui est flagrant sur ces différents graphiques c'est que les chargements ont à peu près la même tendance à différentes heures de la journée, à différents jours de la semaine. En effet, nous voyons très clairement une forme d'arc de cercle/banane des stations au centre. Ces stations sont celles qui possèdent le taux de disponibilité le plus haut. Le fait que ces stations ont un grand taux de disponibilité peu peut être s'expliquer par une utilisation d'autres transports en commun dans ces zones, tel que le métro ou une difficulté à circuler dans ces zones avec un vélib.
Alors que les stations se trouvant au nord et au sud ont quant à elles des taux de remplissage beaucoup plus faible refletant la forte utilisation de vélib dans ces endroits de la ville de Paris.


In [ ]:
#Chargement moyen des stations à 18h
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import plotly.express as px
h = 18
loading_data=loading.to_numpy()
hours = np.arange(h, 168, 24)
load_per_hour = loading_data[:, hours].mean(axis=1)



fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map',
                        color = load_per_hour, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Stations loading - Weekly average at {} h'.format(h))

fig.show()

Nous pouvons représenter les ratios des disponibilités des différentes stations de la ville de Paris sur une carte intéractive. Cette carte nous permet de voir l'emplacement exacte dans la ville des différentes stations. De plus, si l'on clique sur une station, nous voyons directement sa lattitude, sa longitude ainsi que son chargement moyen à 18h.
Nous observons que les stations sont rassemblées en sorte de groupes selon leur chargement. Mise à part certaines stations que nous pouvons interpréter comme étant des outliers, nous voyons que les stations ayant des remplissages similaires forment des regroupements. Nous reviendrons sur l'étude de ces groupes de couleurs formés par les stations lors de la partie sur le clustering.

# Decomposition du data frame loading en jours

Dans un premier temps, nous commençons par créer 7 sous data frame à partir de notre data frame initial loading. On crée un data frame pour chaque jour de la semaine. Les dimensions de 7 sous data frame sont donc de taille (24,1168), car on regroupe les 24h d'une journée à l'intérieur de ce data frame.

In [ ]:
lundi = loading.iloc[:, :24]  
mardi = loading.iloc[:, 24:48] 
mercredi = loading.iloc[:, 48:72] 
jeudi = loading.iloc[:, 72:96] 
vendredi = loading.iloc[:, 96:120] 
samedi = loading.iloc[:, 120:144] 
dimanche = loading.iloc[:, 144:168] 


In [ ]:
df = loading.groupby(loading.columns.str.split('-').str[0], axis=1).mean()

ordre_jours = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam','Dim']
df = df.reindex(columns=ordre_jours)
df.head()

In [ ]:

corr = df.corr( )

# Affichage de la matrice de corrélation
fig, ax = plt.subplots(1, 1)
m = plot_corr_ellipses(corr, ax=ax, cmap='seismic',clim=[0, 1])
cb = fig.colorbar(m)
cb.set_label('Correlation coefficient')
ax.margins(0.1)

Cette figure représente la matrice des corrélation. On observe très clairement une corrélation forte en samedi et dimanche due au fait que ce sont les jours du week end, impliquant potentionnellement une dynamique d'utilisation des vélibs différentes des autres jours de la semaine. D'autres corrélations de cette intensité sont celle entre mardi et mercredi et celle entre jeudi et vendredi.
On voit bien la différence de corrélation entre les jours de la semaine, ceux du week end sont très corrélés entre eux et peu corrélés avec les jours de la semaine. Tandis que, ceux de la semaine sont très corrélés entre eux et peu corrélés avec les jours du week end.
Les vélbs sont utilisés aux mêmes horaires  tous les jours sauf qu'en week end ils sont moins utilisés, ce qui explique que l'on a une corrélation un peu moins forte entre la semaine et le week end.

# Decomposition de lundi en Lundi Nuit et Lundi Journée

A présent, nous allons à partir de chacun des 7 nouveaux sous data frame créer deux nouveaux sous data frame qui seront un data frame contenant les heures de journée et un data frame contenant les heures de nuit. Nous choisissons arbitrairement ces tranches horaires et choisissons comme  de prendre comme convention: 00h à 07h59 et 20h00 à 00h comme heures de nuit et les heures restantes (08h00 à 19h59) comme heures de journée.

In [ ]:
# Sélectionner les colonnes pour les heures de nuit (de 00h à 07h59 et 20h00 à 00h)
lundi_nuit = pd.concat([lundi.iloc[:, 0:8], lundi.iloc[:, 20:24]], axis=1)

# Sélectionner les colonnes pour les heures de journée (de 08h00 à 19h59)
lundi_journée = lundi.iloc[:, 8:20]

# Afficher les premières lignes du dataframe 'lundi_nuit'
print("Lundi Nuit:")
print(lundi_nuit.head())

# Afficher les premières lignes du dataframe 'lundi_journée'
print("\nLundi Journée:")
print(lundi_journée.head())

In [ ]:
#Pour mardi
mardi_nuit = pd.concat([mardi.iloc[:, 0:8], mardi.iloc[:, 20:24]], axis=1)
mardi_journée = mardi.iloc[:, 8:20]

#Pour mercredi
mercredi_nuit = pd.concat([mercredi.iloc[:, 0:7], mercredi.iloc[:, 20:24]], axis=1)
mercredi_journée = mercredi.iloc[:, 8:20]
#Pour jeudi
jeudi_nuit = pd.concat([jeudi.iloc[:, 0:7], jeudi.iloc[:, 20:24]], axis=1)
jeudi_journée = jeudi.iloc[:, 8:20]
#Pour vendredi
vendredi_nuit = pd.concat([vendredi.iloc[:, 0:7], vendredi.iloc[:, 20:24]], axis=1)
vendredi_journée = vendredi.iloc[:, 8:20]
#Pour samedi
samedi_nuit = pd.concat([samedi.iloc[:, 0:7], samedi.iloc[:, 20:24]], axis=1)
samedi_journée = samedi.iloc[:, 8:20]
#Pour dimanche
dimanche_nuit = pd.concat([dimanche.iloc[:, 0:7], dimanche.iloc[:, 20:24]], axis=1)
dimanche_journée = dimanche.iloc[:, 8:20]

print(vendredi_journée)

In [ ]:
# Liste des dataframes à fusionner
dataframes = [lundi_nuit, lundi_journée, mardi_nuit, mardi_journée, mercredi_nuit, mercredi_journée,
               jeudi_nuit, jeudi_journée, vendredi_nuit, vendredi_journée, samedi_nuit, samedi_journée,
               dimanche_nuit, dimanche_journée]

# Fusionner tous les dataframes en un seul dataframe
result = pd.concat(dataframes, keys=['lundi_nuit', 'lundi_journée', 'mardi_nuit', 'mardi_journée', 'mercredi_nuit',
                                     'mercredi_journée', 'jeudi_nuit', 'jeudi_journée', 'vendredi_nuit',
                                     'vendredi_journée', 'samedi_nuit', 'samedi_journée', 'dimanche_nuit',
                                     'dimanche_journée'], axis=1)

# Réinitialiser les index
result.reset_index(inplace=True)
res = result.drop(columns=['index'])

# Afficher le résultat
jours = res.groupby(level=0, axis=1).mean()

# Calcul de la corrélation

corr_jours = jours.corr()

# Affichage de la matrice de corrélation
fig, ax = plt.subplots(1, 1)
m = plot_corr_ellipses(corr_jours, ax=ax, cmap='seismic',clim=[0, 1])
cb = fig.colorbar(m)
cb.set_label('Correlation coefficient')
ax.margins(0.1)


Nous allons à présent nous pencher sur la matrice des corrélations, toutefois, nous prendrons comme variables les horaires de nuit et de journée pour chacun des jours. 
Tout d'abord, nous observons qu'il y a des corrélations fortes entre les différents horaires de journée de la semaine. En effet, on ressent bien que la dynamique d'utilisation des vélibs est similaires durant les horaires de journée au cours des différents jours de la semaine. Cependant, on notera toutefois que l'on retrouve une corrélation assez particulière pour dimanche. En effet, dimnanche journée est très corrélé avec dimanche nuit et très corrélé avec samedi journée et nuit. Nous avions déja expliqué cette corrélation. Le fait qu'il n'y ait pas de grande différence entre les corrélations liées à dimanche nuit et celles liées à dimanche journée. Cela peut s'expliquer par le fait que le dimanche n'est pas considéré comme un jour de travail en France, et que ce jours la les horaires des salariés sont plus allégés dans le cas où ils exercent leur profession.
En dehors de ces jours, nous voyons bien qu'en semaine, les horaires de journée ont une forte corrélation entre eux et une faible corrélation avec les horaires de nuit. Et inversement, les horaires de nuit ont une forte corrélation entre eux et une faible corrélation avec les horaires de journée. Cela est cohérent étant donné qu'au cours de la semaine, la dynamique d'utilisation des vélib est assez similaire étant donné que les horaires de journée sont assez similaires en semaine.

In [ ]:
lundi = categorical_data.iloc[:, :24]  # Sélection des 24 premières colonnes
mardi = categorical_data.iloc[:, 24:48] 
mercredi = categorical_data.iloc[:, 48:72] 
jeudi = categorical_data.iloc[:, 72:96] 
vendredi = categorical_data.iloc[:, 96:120] 
samedi = categorical_data.iloc[:, 120:144] 
dimanche = categorical_data.iloc[:, 144:168]

lundi_nuit = pd.concat([lundi.iloc[:, 0:8], lundi.iloc[:, 20:24]], axis=1)
lundi_journée = lundi.iloc[:, 8:20]
#Pour mardi
mardi_nuit = pd.concat([mardi.iloc[:, 0:8], mardi.iloc[:, 20:24]], axis=1)
mardi_journée = mardi.iloc[:, 8:20]

#Pour mercredi
mercredi_nuit = pd.concat([mercredi.iloc[:, 0:7], mercredi.iloc[:, 20:24]], axis=1)
mercredi_journée = mercredi.iloc[:, 8:20]
#Pour jeudi
jeudi_nuit = pd.concat([jeudi.iloc[:, 0:7], jeudi.iloc[:, 20:24]], axis=1)
jeudi_journée = jeudi.iloc[:, 8:20]
#Pour vendredi
vendredi_nuit = pd.concat([vendredi.iloc[:, 0:7], vendredi.iloc[:, 20:24]], axis=1)
vendredi_journée = vendredi.iloc[:, 8:20]
#Pour samedi
samedi_nuit = pd.concat([samedi.iloc[:, 0:7], samedi.iloc[:, 20:24]], axis=1)
samedi_journée = samedi.iloc[:, 8:20]
#Pour dimanche
dimanche_nuit = pd.concat([dimanche.iloc[:, 0:7], dimanche.iloc[:, 20:24]], axis=1)
dimanche_journée = dimanche.iloc[:, 8:20]
# Liste des dataframes à fusionner
dataframes = [lundi_nuit, lundi_journée, mardi_nuit, mardi_journée, mercredi_nuit, mercredi_journée,
               jeudi_nuit, jeudi_journée, vendredi_nuit, vendredi_journée, samedi_nuit, samedi_journée,
               dimanche_nuit, dimanche_journée]

# Fusionner tous les dataframes en un seul dataframe
result = pd.concat(dataframes, keys=['lundi_nuit', 'lundi_journée', 'mardi_nuit', 'mardi_journée', 'mercredi_nuit',
                                     'mercredi_journée', 'jeudi_nuit', 'jeudi_journée', 'vendredi_nuit',
                                     'vendredi_journée', 'samedi_nuit', 'samedi_journée', 'dimanche_nuit',
                                     'dimanche_journée'], axis=1)

# Réinitialiser les index
result.reset_index(inplace=True)
res = result.drop(columns=['index'])

loading_quali = res.groupby(level=0, axis=1).first()  
loading_quali.head()

# Etude de l'altitude des stations du jeu de donnée.

In [ ]:
#Comparaison du nombre de station en altitude
loading_hill   = loading_data[coord.bonus == 1]
loading_valley = loading_data[coord.bonus == 0]

size = [len(loading_hill), len(loading_valley)]
labels = ['1', '0']

plt.pie(size, labels = labels, autopct="%1.1f%%", 
        colors = [sns.color_palette('tab20b')[-1],sns.color_palette('tab20b')[0]])

plt.show()


Nous voyons sur ce graphique que 89.3% des stations de Paris sont en plaine tandis que 10.7% des stations de Paris sont sur des collines. Il y a ainsi, au sein de la ville de Paris, une majorité de stations étant en plaine.

In [ ]:
#Comparaison du nombre de station en altitude de manière plus visuel (2 Dimensions).
plt.figure(figsize = (8, 8))

sctrplt = plt.scatter(coord['longitude'],coord['latitude'],  
                      c = coord['bonus'], cmap = sns.color_palette('tab20b', as_cmap=True))

plt.xlabel('Longitude', fontsize = 10)
plt.ylabel('Latitude', fontsize = 10)
plt.title('Hilltop stations', fontsize = 15)
plt.legend(handles = sctrplt.legend_elements()[0], labels = ["No hill", "Hill"], fontsize = 10)

plt.show()

Ce graphe nous donne une representation 2D des villes de Paris selon leur altitude. On observe visuellement la forte présente de station de vélib n'étant pas en altitude. On remarque quand même que les stations en alitude forme quelques petits groupes que l'on va retrouver au Nord, à l'Est. Les stations en altitude au Sud sont quant à elles plus dispersés que celles du Nord et de l'Est.

In [ ]:
coord['hill'] = coord['bonus'].astype('category')


fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = "carto-positron",
                        color = 'hill', 
                        color_discrete_map = {0:'midnightblue', 1:'plum'},
                        labels = {0: "hello", 1: "hi"},
                        zoom = 10, opacity = .9,
                        title = 'Hilltop stations')


fig.show()

Cette carte interactive reprend ce qu'on a expliqué précedemment. Cependant, nous pouvons avoir les emplacements exacts. On observe que les stations en alitude se situent vers Montmartre et Belleville. Cela est cohérent quand on connait Paris et on sait que d'après Wikipédia : "Belleville et Montmartre se disputent le point de nivellement le plus haut à Paris" ( point culminant de Paris sur Wikipédia).


In [ ]:
# Filtrer les stations avec Bonus == 1
stations_bonus_1 = coord[coord['bonus'] == 1]

# Afficher les indices et les noms des stations avec Bonus == 1
print("Stations avec Bonus == 1 :")
for index, station in stations_bonus_1.iterrows():
    print("Indice :", index, "- Nom :", station['names'])


In [ ]:
i = 45
loading_data = loading.to_numpy()
print(coord)
n_steps = len(loading_data[i])   # nombre de pas de temps observés
time    = 24*7  # Plage de temps observée


plt.figure(figsize = (20, 6))

plt.plot(loading_data[i])
for j in range(1,7*24+1):
    if j%24==0:
        plt.vlines(x=j,colors='r',ymin=0,ymax=1)
plt.xlabel('Heure')
plt.ylabel('Ratio du nombre de vélib disponible')
plt.title(coord['names'][i+1])

Pour une station en altitude comme Belleville Pré Saint Gervais, nous voyons que le chargement le matin et en journée est assez faible refletant le fait que les citoyens de la ville de Paris utilisent les vélibs en début de journée pour se rendre sur leurs lieux de travail. Le faible chargement de cette station en journée s'explique par le fait que cette station est en altitude et que cela est plus compliqué de redéposer des vélibs dans cette station. En outre on observe des pics de chargement la nuit venant du fait que certaines personnes ont pour rôle de ramener les velibs à leurs stations.

Nous avons mené dans cette partie une analyse descriptive complète au cours de laquelle nous nous somme penchés en détail sur les caractéristiques des stations de la ville de Paris tels que le chargement en vélibs à des heures précise ou au cours de la semaine. Mais aussi, sur l'altitude de ces stations. 
A présent nous allons nous pencher sur une partie importante de notre projet qui est l'Analyse en Composantes Principales (ACP).

# Analyse en ACP.

Au cours de cette partie nous allons nous intérésser à l'analyse en composantes principales qui est une méthode intéréssante dans notre cas au vu des grandes dimensions de nos données. En effet, on sait que cette méthode nous permet de réduire la dimension de nos données tout en gardant une bonne explication (de la variabilité) de nos données. L'objectif principal de l'ACP est de transformer l'ensemble de nos variables, possiblement corrélées, en un ensemble de variable non corrélées appelées composantes principales.
Lorsque l'on se penchera sur nos ACP, nous prendrons soin de normaliser nos données. On a fait ce choix même si les données sont initialement à la même échelle. Si on ne normalise pas, les flèches représentant nos variables sur le cercle des corrélations seront très petites, elles auront une norme petite (signifiant qu'elles ne sont pas corrélées avec nos composantes principales).

In [ ]:
#Réalisation de l'analyse en composantes principales (PCA) sur loading

k=10  ###dimension de ACP

pca = PCA(n_components=k)
load_scale = pd.DataFrame(scale(loading), columns = loading.columns)
load_pca = pca.fit_transform(load_scale)
print(pca.explained_variance_ratio_)
print('-----------------Critère du coude------------------')
plt.plot(range(1, len(pca.explained_variance_) + 1), pca.explained_variance_, marker='o')
plt.xlabel('Nombre de composantes principales')
plt.ylabel('Variance expliquée')
plt.title('Critère du coude')
plt.show()
print('--------------------------------')
print('--- PCA ---')
print('Initial dimension:', loading.shape)
print('Dimension after projection:', load_pca.shape)

print('')

print('--- Explained variance ---')
for i in range(k):    
    print('Component ',i+1,':', round(pca.explained_variance_[i],2), 'i.e.', round(100*pca.explained_variance_ratio_[i],2), '% of the total variance')

loading_data_pca= pd.DataFrame({
    "Dim1" : load_pca[:,0], 
    "Dim2" : load_pca[:,1],
    "names" : coord["names"],
})
print(np.max(load_pca[:,0]))

Nous choisissons arbitrairement la valeur de K et prenons pour cette partie K=10, représentant la dimension de l'ACP. 
Tout d'abord, nous allons utiliser la méthode du coude, cette méthode nous permet de trouver la valeur optimale de K. Pour ce faire, nous allons regarder la figure représentant la variance expliquée en fonction du nombre de composantes principales, et allons nous intéresser au point où il y a une cassure, "un coude". 

Ce graphe nous montre la variance expliquée par chacune des composantes principales, obtenues lors de l'ACP. Nous voyons que la courbe est décroissante, ce qui est normal. En effet, l'objectif de la décomposition en composante principale est de trouver la direction selon laquelle les données varient le plus. Par conséquent, on comprend que la première composante principale est choisie afin de maximiser la variance de nos données lorsqu'elles sont projetées selon cette direction. De ce fait, les composantes principales suivantes expliqueront moins de variance étant données qu'elles sont toutes orthogonales entre elles et donc à la première composante principale qui explique le plus de variance.

Nous observons que la première composante principale, explique 39,81 % des données, la seconde explique 23.5 % et la troisième explique 5.28 %. Nous choisirons K = 5 composantes principales, ce qui nous permettra d'expliquer  près de 76.22% de la variabilité de nos données, tout en ayant réduis nos dimensions de 168 à 5.

In [ ]:

plt.rcParams.update({'font.size': 25})

# Création de la figure et des axes
fig = plt.figure(figsize=(25, 15))
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)

# Visualisation des valeurs propres
ax1.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
ax1.set_xlabel('Composante principale')
ax1.set_ylabel('Variance expliquée')

# Deuxième subplot
ax2.plot(pca.explained_variance_ratio_)
ax2.set_title('Explained variance according to the dimension of the PCA space')
ax2.set_xlabel('Nombre de composantes dans l\'ACP')
ax2.set_ylabel('Variance expliquée')

# Troisième subplot
ax3.scatter(load_pca[:,0], load_pca[:,2], alpha=0.5)
ax3.axhline(y=0, color='r')
ax3.set_title('Graphe des individus avec les composantes 1 et 3')
ax3.set_xlabel('Composante principale 1')
ax3.set_ylabel('Composante principale 3')

# Quatrième subplot
ax4.scatter(load_pca[:,0], load_pca[:,1], alpha=0.5)
ax4.axhline(y=0, color='r')
ax4.set_title('Graphe des individus avec les composantes 1 et 2')
ax4.set_xlabel('Composante principale 1')
ax4.set_ylabel('Composante principale 2')


plt.suptitle('Analyse en composantes principales (PCA)', fontsize=14)
plt.tight_layout()

plt.show()


Les 2 premiers graphiques représentent le pourcentage de variance expliquée selon le nombre de composantes principales. Ces graphiques nous confirment visuellement que nous devons garder K = 5 composantes principales, car au delà le gain de variance expliquée devient minime.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Calculer le remplissage moyen par station
RemplissageMoyenStation = loading.mean(axis=1)

# Définir les catégories de remplissage en fonction de RemplissageMoyenStation
hab = ['faible' if x <= 0.4 else 'moyen' if x <= 0.6 else 'élevé' for x in RemplissageMoyenStation]

# Encoder les étiquettes en valeurs numériques
label_encoder = LabelEncoder()
hab_encoded = label_encoder.fit_transform(hab)

# Créer un dictionnaire de couleurs pour chaque étiquette
color_map = {'faible': 'blue', 'moyen': 'green', 'élevé': 'red'}
colors = [color_map[label] for label in hab]

# Créer un graphique de dispersion des individus colorés en fonction du remplissage moyen
plt.figure(figsize=(8, 6))
plt.scatter(load_pca[:,0], load_pca[:,1], c=colors)
plt.title("Graphes des individus colorés suivant le remplissage moyen", fontsize=25)
plt.xlabel("Composante PC1", fontsize=25)
plt.ylabel("Composante PC2", fontsize=25)
plt.grid(True)
plt.show()

En colorant les individus selon leur taux de remplissage moyen (bleu = < 0.4 ; vert = entre 0.4 et 0.6 ; rouge = > 0.6), on constate que la première composante explique le taux de remplissage moyen des individus au cours de la semaine.

In [ ]:
# Calcul des coordonnées des composantes principales
coord1_1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2_1 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

coord1_2 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2_2 = pca.components_[2] * np.sqrt(pca.explained_variance_[2])

# Création de la figure et des sous-graphiques
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Premier subplot
ax1 = axes[0]
for i, j, nom in zip(coord1_1, coord2_1, loading.columns):
    ax1.text(i, j, nom)
    ax1.arrow(0, 0, i, j, color = 'r', width = 0.0001)
ax1.add_patch(plt.Circle((0, 0), radius=1, color='gray', fill=False))
ax1.axis((-1.2, 1.2, -1.2, 1.2))
ax1.set_title('Composantes 1 et 2')

# Deuxième subplot
ax2 = axes[1]
for i, j, nom in zip(coord1_2, coord2_2, loading.columns):
    ax2.text(i, j, nom)
    ax2.arrow(0, 0, i, j, color = 'r', width = 0.0001)
ax2.add_patch(plt.Circle((0, 0), radius=1, color='gray', fill=False))
ax2.axis((-1.2, 1.2, -1.2, 1.2))
ax2.set_title('Composantes 1 et 3')

plt.show()


Ces graphiques sont les cercles des corrélations, ils représentent chacuns la projections de nos variables sur le plan formé par les composantes 1 et 2 pour le cercle de gauche et par les composantes 1 et 3 pour le cercle de droite. Les composantes d'une station selon les axes sont les coefficients de corrélation entre la variable en question et la composante principale sur laquelle on souhaite projeter la flèche.

Nous allons à présent nous pencher sur le cercle des corrélations des composantes 1 et 2. Tout d'abord, nous observons que les flèches présentent sur ce cercle sont toutes proches de l'extrémité droite du cercle, nous indiquant que les différentes variables ont une corrélation forte avec les deux premières composantes principales de l'ACP. De plus, les variables ont une valeur positive pour l'abscisse, signifiant que les variables et notre première composante principale varient similairement. En outre, on remarque que la très grande majorité de nos variables ont une abscisse étant supérieure à 0.5 et étant en moyenne vers 0.7, cela signifie qu'il y a une corrélation positive et forte de nos variable avec l'axe des abscisse.

Ensuite, nous remarquons que nos variables ont toutefois une ordonnée positive ou négative, contrairement à leurs abscisses qui sont toutes positives. Cependant, il est difficile à l'aide de ce graphe de comprendre à quoi pourrait correspondre cette deuxième composante principale. En effet, le grand nombre de variable de notre jeu de données loading impact l'affichage et donc rend difficile nos interprétations.

De ce fait, pour la suite de cette partie, nous allons nous pencher sur les data frame que nous avons construis au début du projet. En effet, nous nous pencherons d'abord sur le data frame où nous avons regroupé nos colonnes par tranches de 24h pour former des jours comme variable. Ensuite, nous nous pencherons sur celui où nous avons décomposer nos colonnes en horaires de journée et horaires de nuit.


In [ ]:
# Réalisation de l'analyse en composantes principales (PCA) sur df

k=5  ###dimension de ACP


df_scale = pd.DataFrame(scale(df), columns = df.columns)

pca = PCA(n_components=k)
df_scale_pca = pca.fit_transform(df_scale)

print(pca.explained_variance_ratio_)

print('--- PCA ---')
print('Initial dimension:', df.shape)
print('Dimension after projection:', df_scale_pca.shape)

print('')

print('--- Explained variance ---')
for i in range(k):    
    print('Component ',i+1,':', round(pca.explained_variance_[i],2), 'i.e.', round(100*pca.explained_variance_ratio_[i],2), '% of the total variance')

A présent, nous allons prendre une valeur initiale de K valant 5. On remarque bien que les composantes principales ont un pourcentage d'explication de la variance différent que dans le cas où nous avions pris K=10. On rappel que l'on a initialement 7 variables car on utilise le data frame que l'on a crée contenant les 7 jours de la semaine comme variable. On a ici une réduction du nombre de dimensions de 7 à 5.

In [ ]:

plt.rcParams.update({'font.size': 15})

# Visualisation des valeurs propres
plt.figure(figsize=(25, 15))

plt.subplot(2, 2, 1)
plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
plt.xlabel('Composante principale')
plt.ylabel('Variance expliquée')

# Deuxième subplot
plt.subplot(2, 2, 2)
plt.plot(pca.explained_variance_ratio_)
plt.title('Explained variance according to the dimension of the PCA space')
plt.xlabel('Nombre de composantes dans l\'ACP')
plt.ylabel('Variance expliquée')

# Troisième subplot
plt.subplot(2, 2, 3)
plt.scatter(df_scale_pca[:,0], df_scale_pca[:,2], alpha=0.5)
plt.axhline(y = 0, color = 'r')
plt.title('Graphe des individues avec les composantes 1 et 3')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 3')

# Quatrième subplot
plt.subplot(2, 2, 4)
plt.scatter(df_scale_pca[:,0], df_scale_pca[:,1], alpha=0.5)
plt.axhline(y = 0, color = 'r')
plt.title('Graphe des individues avec les composantes 1 et 2')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')

plt.suptitle('Analyse en composantes principales (PCA)', fontsize=14)
plt.tight_layout()

plt.show()



In [ ]:

# Calcul des coordonnées des composantes principales
coord1_1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2_1 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

coord1_2 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2_2 = pca.components_[2] * np.sqrt(pca.explained_variance_[2])

# Création de la figure et des sous-graphiques
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Premier subplot
ax1 = axes[0]
for i, j, nom in zip(coord1_1, coord2_1, df.columns):
    ax1.text(i, j, nom)
    ax1.arrow(0, 0, i, j, color = 'r', width = 0.0001)
ax1.add_patch(plt.Circle((0, 0), radius=1, color='gray', fill=False))
ax1.axis((-1.2, 1.2, -1.2, 1.2))
ax1.set_title('Composantes 1 et 2')

# Deuxième subplot
ax2 = axes[1]
for i, j, nom in zip(coord1_2, coord2_2, df.columns):
    ax2.text(i, j, nom)
    ax2.arrow(0, 0, i, j, color = 'r', width = 0.0001)
ax2.add_patch(plt.Circle((0, 0), radius=1, color='gray', fill=False))
ax2.axis((-1.2, 1.2, -1.2, 1.2))
ax2.set_title('Composantes 1 et 3')

plt.show()


Nous voyons sur le cercle de corrélation de gauche (avec les composantes 1 et 2) une symétrie de certains jours de la semaine par rapport au première axe. En effet, les jours Lundi, Mardi, Mercredi et Jeudi sont très proches. A l'inverse nous voyons que les jours du week end sont eux aussi très proches mais de l'autre côté de l'axe des abscisses. Nous pouvons en déduire que l'axe formé par la deuxième composante principale permet de mettre en évidence la différence entre les jours de la semaine et du week end. Le jour de la semaine vendredi se trouvant entre les deux blocs de jours ( celui formé par les autres jours de la semaine et celui formé par les jours du week end), peut s'expliquer par le fait que ce jour est un jour assez proche du week end. En effet, nous observons que les jours lundi et vendredi sont plus proches des variables des jours du week end puisque ce sont les jours de la semaine qui précède ( pour vendredi) et suit(pour lundi) le week end. A l'inverse, le jour mercredi étant le plus éloigné du week end car en milieu de semaine, se traduit sur le cercle des corrélations, par le fait que la variable mercredi est celle qui est la plus éloignée des variables samedi et dimanche.

In [ ]:
k=5  ###dimension de ACP

jours_scale = pd.DataFrame(scale(jours), columns = jours.columns)

pca = PCA(n_components=k)
jours_scale_pca = pca.fit_transform(jours_scale)

print(pca.explained_variance_ratio_)

print('--- PCA ---')
print('Initial dimension:', jours_scale.shape)
print('Dimension after projection:', jours_scale_pca.shape)

print('')

print('--- Explained variance ---')
for i in range(k):    
    print('Component ',i,':', round(pca.explained_variance_[i],2), 'i.e.', round(100*pca.explained_variance_ratio_[i],2), '% of the total variance')


Dans cette ACP nous allons utilisé le data frame contenant les variables journée et nuit pour chaque jours de la semaine, d'où le fait que nous ayons 14 variables initialement ( 2 pour chacun des jours de la semaine). 

In [ ]:
plt.rcParams.update({'font.size': 15})

# Visualisation des valeurs propres
plt.figure(figsize=(25, 15))

plt.subplot(2, 2, 1)
plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
plt.xlabel('Composante principale')
plt.ylabel('Variance expliquée')

# Deuxième subplot
plt.subplot(2, 2, 2)
plt.plot(pca.explained_variance_ratio_)
plt.title('Explained variance according to the dimension of the PCA space')
plt.xlabel('Nombre de composantes dans l\'ACP')
plt.ylabel('Variance expliquée')

# Troisième subplot
plt.subplot(2, 2, 3)
plt.scatter(jours_scale_pca[:,0], jours_scale_pca[:,2], alpha=0.5)
plt.axhline(y = 0, color = 'r')
plt.title('Graphe des individues avec les composantes 1 et 3')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 3')

# Quatrième subplot
plt.subplot(2, 2, 4)
plt.scatter(jours_scale_pca[:,0], jours_scale_pca[:,1], alpha=0.5)
plt.axhline(y = 0, color = 'r')
plt.title('Graphe des individues avec les composantes 1 et 2')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')

plt.suptitle('Analyse en composantes principales (PCA)', fontsize=14)
plt.tight_layout()

plt.show()


In [ ]:

# Calcul des coordonnées des composantes principales
coord1_1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2_1 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

coord1_2 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2_2 = pca.components_[2] * np.sqrt(pca.explained_variance_[2])

# Création de la figure et des sous-graphiques
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Premier subplot
ax1 = axes[0]
for i, j, nom in zip(coord1_1, coord2_1, jours.columns):
    ax1.text(i, j, nom)
    ax1.arrow(0, 0, i, j, color = 'r', width = 0.0001)
ax1.add_patch(plt.Circle((0, 0), radius=1, color='gray', fill=False))
ax1.axis((-1.2, 1.2, -1.2, 1.2))
ax1.set_title('Composantes 1 et 2')

# Deuxième subplot
ax2 = axes[1]
for i, j, nom in zip(coord1_2, coord2_2, jours.columns):
    ax2.text(i, j, nom)
    ax2.arrow(0, 0, i, j, color = 'r', width = 0.0001)
ax2.add_patch(plt.Circle((0, 0), radius=1, color='gray', fill=False))
ax2.axis((-1.2, 1.2, -1.2, 1.2))
ax2.set_title('Composantes 1 et 3')

plt.show()


Nous rappelons que pour les variables nous avons pris comme convention : 
- horaires de nuit : 00h à 7h59 et 20h00 à 00h
- horaires de journée : 8h00 à 19h59

Pour la première composante principale on reprend ce qu'on a dis pour la première ACP (cette composante explique le chargement moyen en vélibs des stations).

La deuxieme composante principale semble porter l'information de la plage horaire jour/nuit. En effet, les variables journée ( ne contenant pas les horaires de nuit) sont corrélées positivement avec la deuxième composante principale, tandis que, les variables nuit contenant les heures de nuit sont corrélées négativement avec la deuxième composante principale.  

Maintenant que nous avons bien vu et compris ce qu'était l'ACP, nous allons à présent nous pencher sur les différentes méthodes de clusering.

# MCA

### Création de Contingency Table et CA

In [ ]:
import prince
from prince import MCA
from prince import CA

On va commncer avec une analyse de correspondance entre la variable de jours et la variable de loading. Vu que cette dernière est quantitative, il faut la discretiser.

In [ ]:
loading.mean(axis=1).describe()

On peut définir une liste d'intervalles, qui contient des valeurs seuilles pour découper la gamme de données quantitatives de loading en segments distincts. On va utiliser les quantiles de loading moyen pour definir ces intervalles.

In [ ]:
intervals = [0, 0.204574, 0.353352, 0.532460, 0.7, 1.]
labels = ['très peu chargé', 'peu chargé', 'moyen', 'chargé', 'très chargé']

# Appliquer la transformation
load_quali = pd.DataFrame()
for column in jours.columns:
    load_quali[column] = pd.cut(jours[column], bins=intervals, labels=labels, right=False)


load_quali.head(6)

On a utilisé les trois quartiles pour construire les 3 premiers intervalles. Ensuite, on a ajouté un seuil intermidiare entre $q_{75}$ et 1 qui vaut 0.7 pour considerer les stations qui sont trés chargés.

In [ ]:
jours.head()

In [ ]:
# Sélection des colonnes à considérer
columns_to_consider = ['dimanche_nuit', 'dimanche_journée', 'jeudi_nuit', 'jeudi_journée', 
        'lundi_nuit', 'lundi_journée', 'mardi_nuit', 'mardi_journée', 
          'mercredi_nuit', 'mercredi_journée', 'samedi_nuit', 'samedi_journée', 
          'vendredi_nuit', 'vendredi_journée']
# Création de la table de contingence
contingency_table = pd.DataFrame()

# Pour chaque colonne, compter le nombre d'occurrences de chaque modalité
for col in columns_to_consider:
    contingency_table[col] = load_quali[col].value_counts()

# Affichage de la table de contingence
contingency_table.head()

Ensuite, on peut construire à partir du dernier data frame, la contigency table entre la variable jour et la variable de loading discrétisé.

In [ ]:

ansco = CA(n_components=4)

# Ajustement du modèle aux données encodées
ansco.fit(contingency_table)

# Affichage des résultats
print(ansco.eigenvalues_summary)
print(ansco.row_coordinates(contingency_table))
print(ansco.column_coordinates(contingency_table))
print(ansco.total_inertia_)

# Récupérer les coordonnées des lignes et des colonnes
row_coords = ansco.row_coordinates(contingency_table)
col_coords = ansco.column_coordinates(contingency_table)

# Plot des coordonnées des lignes (modalités de chargement)
plt.figure(figsize=(10, 6))
plt.scatter(row_coords[0], row_coords[1], label='Modalités de chargement')

# Plot des coordonnées des colonnes (jours de la semaine)
plt.scatter(col_coords[0], col_coords[1], label='Jours de la semaine')

# Ajouter des labels aux points
for i, txt in enumerate(contingency_table.index):
    plt.annotate(txt, (row_coords[0][i], row_coords[1][i]))

for i, txt in enumerate(contingency_table.columns):
    plt.annotate(txt, (col_coords[0][i], col_coords[1][i]))

# Ajouter une légende
plt.legend() 

# Ajouter des labels aux axes
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')

# Afficher le scatter plot
plt.title('Scatter plot des modalités de chargement et des jours de la semaine')
plt.show()


On voit que les 4 premiers axes expliquent 100% de données. On observe sur ce graphe que la plupart des périodes de la semaine sont proches d'une modalité de chargement, à l'exception de "lundi_journée". On peut en déduire qu'aucune tendance générale de chargement moyen ne se dégage pour le lundi entre 8h et 20h.

## Lien avec la variable bonus

In [ ]:
load_quali['Hilltop']=coord['bonus']
nan_count = load_quali['Hilltop'].isna().sum()
print("Nombre de NaN dans la colonne 'Hilltop' :", nan_count)

load_quali.head(5)

In [ ]:
load_quali['Hilltop'].fillna(0, inplace=True)
load_quali['Hilltop'] = load_quali['Hilltop'].fillna(0).astype(bool)
load_quali.head(5)

Nous approfondirons notre analyse en intégrant également la variable bonus pour examiner son association avec la modalité hilltop. Nous avons observé que l'ajout de la colonne hilltop entraînait la conversion de la première valeur en NaN, que nous avons rectifiée en la rétablissant à sa valeur initiale.

In [ ]:
# Analyse en composantes principales multiples (MCA)
mca = prince.MCA(n_components=5)
mca.fit(load_quali) 

display(mca.eigenvalues_summary)

mca.scree_plot()

mca.plot(
    load_quali,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=False
)

def plot_mca(ax1=0, ax2=1, mca=mca, data=load_quali):
    dataset = mca.transform(data)
    dataset.reset_index(inplace=True)
    sns.scatterplot(data = dataset,
                  x = ax1, y = ax2
                , alpha=.7)

    plt.xlabel('Component {} — {:.2f}%'.format(ax1, mca.percentage_of_variance_[ax1]))
    plt.ylabel('Component {} — {:.2f}%'.format(ax2, mca.percentage_of_variance_[ax2]))
    plt.grid(True)
    plt.show()

plot_mca(0,1)

Lors de l'application de MCA nous constatons que les deux premiers axes expliquent seulement environ 20% des données, ce qui est relativement faible. Par conséquent, les conclusions tirées à partir de cette MCA pourraient manquer de fiabilité dans ce contexte.

In [ ]:
# Plot des coordonnées des modalités
mca.plot(
    load_quali,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=False,
    show_column_labels=False,
    show_row_labels=False
)

On a projeté les modalités sur l'espace par les deux composantes principales trouvées par MCA. On voit que les modalités "Hilltop_False" et "Hilltop_True" correspondent aux points de coordonnés (0,0) et (0,0.12). Vu que les deux modalités sont trop proches et que les deux premiers axes n'expliquent que 20% de données, on peut confirmer que la projection dans ce cas n'est pas fiable.

# K-means


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import colors
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

Dans cette partie, nous allons nous pencher sur les méthodes de clustering tel que k-means, GMM et CAH.Le but du clustering sera de regrouper les stations dans des groupes en fonctions des tendances de chargement sur une semaine.

Dans un premier temps nous allons nous intérésser à la méthode k-means. La méthode k means permet de partitionner notre ensemble de données en K groupes, aussi appelés clusters. Ces groupes se formeront de la manière suivantes. Nous considérons K points au hasard dans nos données qui seront considérés comme étant les centroïdes initiaux. Ensuite, l'algorithme se chargera de placer chaque données dans les clusters correspondants, celui qui minimise la variance intra classe pour nos données. Il y aura des nouveaux calculs de centroides et donc des répartitions encore différentes des données dans nos K clusters, et ce processus se fera jusqu'à converger vers une solution.

L'objectif de cette première méthode est de regrouper nos données en K groupes homogènes, où les données au sein de chaque groupe sont similaires entre elles mais différentes des autres clusters (inertie intra-classe faible, inertie inter-classe forte).

In [ ]:
load_reduce = load_pca[:,:3]
load_data=loading.to_numpy()

In [ ]:
K=7
kmeans_pca= KMeans(n_clusters=K,init='k-means++')
clusters_pca = kmeans_pca.fit_predict(load_reduce)

cmap = plt.get_cmap('Set3',K)
plt.bar(*np.unique(clusters_pca, return_counts=True), color=cmap.colors)
plt.ylabel("Frequency")
plt.show()
clusters_pca

On fixe k-means++ comme initialisation vu que cette méthode choisit les centroïdes de manière stratégique pour favoriser une convergence plus rapide et des clusters de meilleure qualité. Au lieu de sélectionner les centroïdes de façon aléatoire, k-means++ utilise une approche probabiliste qui privilégie les points éloignés les uns des autres. Cela contribue à réduire les risques de convergence vers un minimum local et améliore la qualité globale des clusters.

Nous utiliserons la méthode k means ++ qui est une amélioration de la méthode k means car elle place les centroïdes initiaux de manière plus intelligente afin de mieux observer la structure de nos données et de converger vers une solution globale plus rapidement.

Tous d'abord, nous commençons par choisir un nombre K=7 de clusters. 
Ce graphique nous montre l'effectif des différents clusters obtenus après avoir utiliser k means. Globalement, l'effectifs des clusters est homogènes, entre 100 et 250 pour chaque cluster.

In [ ]:
cmap = plt.get_cmap('Set3',K)
kmeans_pca= KMeans(n_clusters=K,init='k-means++')
clusters_pca = kmeans_pca.fit_predict(load_reduce)
n_pixel_x = 41
n_pixel_y = 29
print(clusters_pca.shape)
load_image = clusters_pca.reshape((n_pixel_x, n_pixel_y))

# Visualisation des clusters
plt.scatter(load_pca[:, 0], load_pca[:, 1], c=clusters_pca, cmap='viridis')
plt.colorbar(label='Cluster')
plt.title('Visualisation des clusters avec KMeans')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.show()

Nous affichons ici le graphe des individus représentant la répartition des données au sein de nos 7 clusters dans le plan de l'ACP formé par les deux premières composantes. Ce graphe confirme bien les résultats obtenus précédemment, en effet, on note que les clusters ont des effectifs assez similaires.

In [ ]:

x = np.arange(0, 168)
for i in range(K):
    plt.plot(x, np.mean(load_data[clusters_pca == i], axis = 0), color=cmap.colors[i])
    plt.title("Chargement moyen de la classe "+str(i+1))
    plt.xlabel("heure")
    plt.ylabel("Loading")

    plt.show()

Nous affichons le chargement moyen des données selon leur cluster, en fonction de l'heure.

On observe des tendances différentes pour chaque clusters. Les pics de remplissage sont généralement soit en pleine journée soit la nuit. Certains clusters sont formés par des stations peu remplies en week-end tandis que d'autres sont formés par des stations utilisées tous les jours de la semaine. On constate également une certaine périodicité entre les jours de la semaine (pics toujours au même moment pour un cluster).

In [ ]:
kmeans = KMeans(init='k-means++', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans, k=(1,11))

visualizer.fit(load_reduce)
visualizer.show()

Lors de l'implémentation de k means précédente, nous avons choisi arbitrairement la valeur de K. Cependant, dans l'optique d'obtenir une valeur optimale du nombre de cluster K pour l'implémentation de k means, nous allons utiliser la méthode du Coude. Cette méthode nous permet d'obtenir la variance intra classe en fonction du nombre de cluster, le nombre optimal de cluster K se lira à l'endroit où il y a la présence d'une cassure sur notre courbe. Les graphe ci représente la variance intra classe en fonction du nombre de cluster, la méthode du coude nous permet donc d'y lire le nombre optimal de cluster K qui est ici K=3.
Nous avons cependant choisi un nombre de cluster K = 5. Ce choix va être motivé dans la suite.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(15,8))

for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2)
    
    # Create SilhouetteVisualizer instance with KMeans instance
    visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(load_pca)

Le score silhouette correspond à la différence entre la séparation et la cohésion que l'on normalise par le maximum (entre la séparation et la cohésion). Plus le score silhouette est proche de 1 et mieux la classification est, tandis que, si le score silhouette est négatif, cela signifie qu'il y a une mauvais classification. On souhaite avoir un score silhouette proche de 1 étant donné que notre but est de maximiser la variance inter classe, ici la séparation des clusters, et de minimiser la variance intra classe, ici la cohésion des clusters.

Ces graphiques représente les graphes du score silhouette de chacun des points pour différents nombre de clusters K. 
Dans notre cas, comme nous avons choisi précédemment le nombre optimal de cluster (K=5), nous analyserons seulement ce graphique. Sur cette figure nous voyons que les différents formes (une pour chaque cluster) ont une épaisseur différentes cela s'explique par la proportion d'individus des clusters. La flèche en pointillé rouge représente le coefficient silhouette moyen des individus, valant dans notre cas 0.24. Tous les clusters dépassant cette ligne en pointillé rouge nous indiquant que nous avons un bon clustering (une bonne cohésion et une bonne séparation). 

Les pics négatifs s'expliquent par le fait qu'il y a des chevauchements entre les points de différents clusters, de ce fait le coefficient silhouette est négatif puisque que certains points (modélisant les stations) sont plus proches des points présents dans les autres clusters. En outre, cela est inévitable au vu de la forme de nos données, nous ne pouvons pas éviter d'avoir des chevauchements entre les classes sur leurs bords.

In [ ]:
K=5
cmap = plt.get_cmap('Set3',K)
kmeans_pca= KMeans(n_clusters=K,init='k-means++')
clusters_pca = kmeans_pca.fit_predict(load_reduce)


# Visualisation des clusters
plt.scatter(load_pca[:, 0], load_pca[:, 1], c=clusters_pca, cmap='viridis')
plt.colorbar(label='Cluster')
plt.title('Visualisation des clusters avec KMeans')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.show()


Dans un premier temps, nous avons utiliser l'algorithme k means ++ pour obtenir nos clusters et avons projeté nos données sur un espace de dimension réduit à l'aide de l'ACP. Cette figure, nous donne une représentation des 5 clusters obtenus à l'aide de l'algorithme k means dans un plan formé par les deux premières composantes principales obtenus lors de l'ACP. Nous observons que nos clusters forment des groupes bien distincts. De plus nous observons que les clusters ont une proportion de stations assez similaire comme vu précédemment sur le graphe silhouette.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic


table = pd.crosstab(coord["bonus"], clusters_pca)

# Affichez la table de contingence
print(table)

# Tracez un mosaic plot en utilisant la bibliothèque statsmodels.graphics.mosaicplot
plt.figure(figsize=(10, 6))
mosaic(table.stack(), title='Mosaic Plot entre les classes et la variable Bonus pour les collines')
plt.show()

Les stations situées sur une colline sont globalement toutes concentrés dans 2 clusters. On peut supposer que ces stations partage la même tendance. Cependant, les clusters contenant des stations en altitude contiennent aussi d'autres stations. On peut en déduire que les stations en altitude n'ont pas de tendances exclusives.

In [ ]:
x = np.arange(0, 168)
for i in range(K):
    plt.plot(x, np.mean(load_data[clusters_pca == i], axis = 0), color=cmap.colors[i])
    plt.title("Chargement moyen de la classe "+str(i))
    plt.xlabel("heure")
    plt.ylabel("Loading")

    plt.show()

Nous affichons le chargement moyen des données selon leur cluster, en fonction de l'heure.

Comme pour le cas avec 7 clusters, on observe des tendances différentes pour chaque clusters. Les pics de remplissage sont généralement soit en pleine journée soit la nuit. Certains clusters sont formés par des stations peu remplies en week-end tandis que d'autres sont formés par des stations utilisées tous les jours de la semaine. On constate également une certaine périodicité entre les jours de la semaine (pics toujours au même moment pour un cluster).

### Différences de clustering effectué avec les données traités par l'ACP et données brutes

In [ ]:
h = 30 

hours = np.arange(h, 168, 24)
cluster = clusters_pca

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map',
                        color = cluster, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Clustering ACP des stations avec données traités par l\'ACP ')

fig.show()

Nous voyons que certains clusters correspondent à des zones géographiques précisent tandis que d'autres sont plus éparses. Nous pouvons également remarquer que certaines zones sont mélangés entre plusieurs clusters (par ex. au centre ville). Ces chevauchements entre clusters peuvent s'expliquer par le fait que certaines stations ont des profiles assez similaires, de ce fait, il n'est pas évident de savoir à quel cluster associer ces stations.

### Confirmation du choix de la dimension 

Pour chaque classe, nous allons :
- Créer 4 groupes de 45 individus tirés aléatoirement
- Afficher le chargement moyen de ces groupes

Buts :
- Vérifier que les clusters sont homogènes
- Visualiser le comportement de chaque classe

In [ ]:
p = 45  # nb d'individues
n=4  #nb de groupes
plt.figure(figsize=(12, 8))

# Boucle sur les clusters
for cluster in range(5):
    
    indices_cluster = np.where(clusters_pca == cluster)[0]
    
    for i in range(n):
        # Sélection de p indices aléatoires
        np.random.shuffle(indices_cluster)
        indices_to_plot = indices_cluster[:p]
        
        # Calcul de la moyenne des chargements pour ces individus
        mean_loading = np.mean(loading.iloc[indices_to_plot], axis=0)
        
        # Tracé de la moyenne des chargements
        plt.plot(mean_loading, label=f"Cluster {cluster}, Group {i+1}", marker='o')


    # Ajouter des légendes et des étiquettes
    plt.xlabel("Composantes principales")
    plt.ylabel("Loading moyen")
    plt.title("Loading moyen pour classe " + str(cluster))
    plt.legend()
    plt.grid(True)
    plt.show()

Nous avons choisi 4 échantillons de 45 individus pour chaque clusters et avons pris soin de comparer le chargement moyen de ces 4 échantillons d'individus. Nous voyons bien que les distributions sont très supperposées, nous confirmant l'homogénéité des chargements moyen des individus au sein de chaque cluster. Cependant, un des clusters semble avoir des chargements moyen plus erratique entre ses stations. On peut l'expliquer par le fait que l'amplitude de chargement moyen est plus faible. De ce fait, cela nous permet de confirmer que le choix de K=5 clusters semble cohérent.

# Classification Ascendante Hiérarchique

Dans cette partie nous allons nous pencher sur la Classification Ascendante Hierarchique (CAH) qui est une méthode nous permettant de regrouper nos données en clusters hierarchiques. Nous nous appuierons sur nos résultats afin d'obtenir une représentation visuelle à l'aide des dendrogrammes.

In [ ]:
#hierarchical ascending classification (CAH)
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

d = pdist(loading, metric="euclidean")

# Clustering

hclustcomplete = linkage(d, method="complete")
hclustsingle = linkage(d, method="single")
hclustaverage = linkage(d, method="average")
hclustward = linkage(d, method="ward")

# Visualisation des dendrogrammes
plt.figure(figsize=(10, 10))

plt.subplot(4, 1, 1)
dendrogram(hclustsingle, labels=None)
plt.title("Single Linkage")
plt.xlabel("Observations")
plt.ylabel("Distance")

plt.subplot(4, 1, 2)
dendrogram(hclustcomplete, labels=None)
plt.title("Complete Linkage")
plt.xlabel("Observations")
plt.ylabel("Distance")
#representation

plt.subplot(4, 1, 3)
dendrogram(hclustaverage, labels=None)
plt.title("Average Linkage")
plt.xlabel("Observations")
plt.ylabel("Distance")

plt.subplot(4, 1, 4)
dendrogram(hclustward, labels=None)
plt.title("ward Linkage")
plt.xlabel("Observations")
plt.ylabel("Distance")

plt.tight_layout()
plt.show()


Nous allons tout d'abord nous pencher sur 4 méthodes de calculs de similarités entre clusters que sont single, complete, average linkage et ward, nous permettant de savoir à quels moments fusionner nos clusters. La méthode de calcul single linkage est une méthode mesurant la plus petite distance entre les points de deux clusters.

La méthode de calcul complete linkage est une méthode mesurant la plus grande distance entre les points de deux clusters.

La méthode de calcul average linkage est une méthode mesurant la distance moyenne entre les couples de points des deux clusters. Pour ce faire on va considérer un point dans un cluster, calculer les distances entre ce point et tous les autres points du deuxième cluster, calculer la distance moyenne et répeter cela pour tous les autres points.

La méthode de calcul Ward est une méthode cherchant à fusionner les clusters en minimisant la variance intra-cluster.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Liste des liaisons à tester
linkages = ['ward', 'complete', 'average', 'single']

# Création de sous-graphiques
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

# Boucle sur chaque liaison
for i, linkage in enumerate(linkages):
    # Initialisation de l'algorithme de clustering
    cah = AgglomerativeClustering(linkage=linkage)
    visualizer = KElbowVisualizer(cah, k=(1, 11), ax=axes[i])
    
    # Ajustement des données et affichage
    visualizer.fit(load_reduce)
    visualizer.set_title(f'Elbow Method for linkage={linkage}')

# Affichage global
plt.tight_layout()
plt.show()

Avec la méthode du coude, le nombre idéal de cluster pour Ward est K=4. Nous prendrons donc K=4 clusters pour cette méthode.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns

# Création des quatre sous-graphiques
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
n_clusters = 4

# Méthodes de liaison à utiliser
linkages = ['single', 'average', 'complete', 'ward']

# Boucle sur les sous-graphiques et les méthodes de liaison
for i, linkage in enumerate(linkages):
    # Création du modèle de CAH avec le nombre de clusters désiré
    cah = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
    
    # Effectuer la classification ascendante hiérarchique sur les données PCA
    clusters = cah.fit_predict(load_pca)
    
    # Visualisation des clusters
    sns.scatterplot(x=load_pca[:, 0], y=load_pca[:, 1], hue=clusters, palette='viridis', legend='full', ax=axs[i//2, i%2])
    axs[i//2, i%2].set_title(f'CAH avec liaison {linkage.capitalize()}')
    axs[i//2, i%2].set_xlabel('Composante Principale 1 (PC1)')
    axs[i//2, i%2].set_ylabel('Composante Principale 2 (PC2)')

plt.tight_layout()
plt.show()


Nous avons affiché le graphe des individus pour chacune des 4 méthodes de claculs expliquées précédemment. Tout d'abord, nous voyons que les clusters ne sont pas du tout différenciable et bien représenté pour la méthode single linkage. Cela vient du fait que lorsque nous avons considérer le saut le plus haut pour couper notre dendrogramme, les clusters n'étaient pas homogènes dans leurs répartions. Les 3 autres méthodes de calculs ont quant à elles des clusters assez similaires. Cependant, on notera que l'on retrouvera plus de chevauchements pour les méthodes average et complete linkage que pour Ward. La meilleure méthode de calcul parmis les 4 semble donc être la méthode Ward linkage.

In [ ]:
# Créer un objet CAH avec le nombre de clusters désiré
n_clusters = 4
linkage='ward'
cah = AgglomerativeClustering(n_clusters=n_clusters,linkage=linkage)

# Effectuer la classification ascendante hiérarchique sur les données après ACP (load_pca)
clusters_ward = cah.fit_predict(load_pca)


plt.figure(figsize=(10, 6))
sns.scatterplot(x=load_pca[:,0], y=load_pca[:,1], hue=clusters_ward, palette='viridis', legend='full')
plt.title('Classification Ascendante Hiérarchique (CAH) avec un le lien '+ linkage)
plt.xlabel('Composante Principale 1 (PC1)')
plt.ylabel('Composante Principale 2 (PC2)')
plt.show()


Cette figure représente la projection de nos données sur le plan formé par nos deux premières composantes principales. Nous voyons que nos 4 clusters sont globalement bien différenciables, même si on observe de légers chevauchements entre les clusters. Nous allons comparer cette méthode par la suite avec les autres méthodes de calcul.

In [ ]:
h = 30

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map',
                        color = clusters_ward, color_continuous_scale = px.colors.sequential.Plasma_r, 
                        zoom  = 10, opacity = 1,
                        title = 'Clustering des stations avec liason Ward')

fig.show()

Un des clusters est facilement identifiable au centre de Paris. Un autre occupe une bonne partie du nord de la ville. Les autres zones semblent équiréparties entres les 4 clusters.

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic


table = pd.crosstab(coord["bonus"], clusters_ward)

# Affichez la table de contingence
print(table)

# Tracez un mosaic plot 
plt.figure(figsize=(10, 6))
mosaic(table.stack(), title='Mosaic Plot entre les classes et la variable Bonus pour les collines')
plt.show()

Les stations en altitude appartiennent presque toutes au même cluster. On peut en déduire qu'elles présentent toutes une tendance de chargement similaire. Toutefois, ce cluster contient également des stations situées en plaines. La tendance de chargement des stations en altitude n'est donc pas exclusive à ces stations.

### Confirmation de la dimension

In [ ]:
p = 45  # nb d'individues
n= 4  #nb de groupes
plt.figure(figsize=(12, 8))

# Boucle sur les clusters
for cluster in range(4):
    
    indices_cluster = np.where(clusters_ward == cluster)[0]
    
    for i in range(4):
        # Sélection de p indices aléatoires
        np.random.shuffle(indices_cluster)
        indices_to_plot = indices_cluster[:p]
        
        # Calcul de la moyenne des chargements pour ces individus
        mean_loading = np.mean(loading.iloc[indices_to_plot], axis=0)
        
        # Tracé de la moyenne des chargements
        plt.plot(mean_loading, label=f"Cluster {cluster}, Group {i+1}", marker='o')


    # Ajouter des légendes et des étiquettes
    plt.xlabel("Composantes principales")
    plt.ylabel("Loading moyen")
    plt.title("Loading moyen pour classe " + str(cluster))
    plt.legend()
    plt.grid(True)
    plt.show()

On observe une homogénéité dans les chargements d'un cluster. Cependant, un des clusters semble avoir une plus grande variabilité dans les chargements de ses stations.

Maintenant que nous avons vu la méthode CAH, nous allons nous intérésser à notre troisième et dernière méthode de clustering, GMM.

# Gaussian Mixture Model

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score 

Dans cette partie, nous nous intéressons à l'approche du mélange de modèles gaussiens, qui est une méthode utilisée en clustering mais assez différentes de la méthode k-means. En effet, pour la méthode k means nous imposions à chaque point d'appartenir à un cluster tandis que pour la Gaussian Mixture Model, nous allons attribuer à chacun des points une probabilités d'appartenance à chaque cluster. Ensuite, le point sera attibué au cluster pour lequel la probabilité d'appartenance est la plus grande.

Cette méthode est utilisée pour le clustering des variables quantitatives. En effet, elle suppose que notre échantillon contient plusieurs distributions gaussiennes différentes, c'est-à-dire que chaque cluster suit une loi gaussienne définie par des paramètres différents des autres clusters. La méthode cherche alors à trouver ces paramètres en estimant la moyenne et la matrice de covariance empiriques de chaque clusters et ensuite maximisant la vraisemblance des données observées sous le modèle sous plusieurs itérations avec l'algorithme de EM.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 12))
n_clusters = 3
liste = ['full', 'tied', 'diag', 'spherical']

# Parcourir la liste des types de covariance
for i, cov_type in enumerate(liste):
    # Créer un modèle GMM avec le type de covariance actuel
    gmm = GaussianMixture(n_components=n_clusters, covariance_type=cov_type, n_init=10, init_params='k-means++')
    clusters_gmm = gmm.fit_predict(load_pca)
    
    # Trouver les coordonnées des sous-graphiques
    row = i // 2
    col = i % 2
    
    # Afficher le scatterplot sur le sous-graphique correspondant
    sns.scatterplot(x=load_pca[:, 0], y=load_pca[:, 1], hue=clusters_gmm, palette='viridis', legend='full', ax=axs[row, col])
    axs[row, col].set_title(f'GMM Clustering with {cov_type} covariance')
    axs[row, col].set_xlabel('Composante Principale 1 (PC1)')
    axs[row, col].set_ylabel('Composante Principale 2 (PC2)')

plt.tight_layout()
plt.show()

Dans cette partie nous nous sommes intéréssés aux différentes structures de covariance des composantes gaussiennes que nous utilisons pour la modélisation de nos clusters. Tout d'abord, nous voyons que la méthode 'tied' est la moins adaptée, ici, nous ne parvenons pas à identifier les clusters car ces derniers sont superposés. Ensuite, les méthodes 'diag' et 'full' présentent certes de meilleurs résultats car nous parvenons à identifier plus facilement les clusters que pour la méthode 'tied'. Cependant, nous pouvons constater que les clusters n'ont pas une répartition homogène et qu'il y a un chevauchement assez notable des points aux frontières des clusters. Enfin, la méthode 'sphérical' apparait comme étant la meilleur dans notre cas, puisque les clusters ont une répartition plus homogène que pour les autres méthodes et il y a certes de très légers chevauchements mais cela peut s'expliquer par la représentation.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Nombre maximal de clusters à tester
k_max = 15

# Initialisation des listes pour BIC et silhouette
bic = []
silhouette = []

# Boucle pour tester différents nombres de clusters
for k in range(2, k_max):
    # Modèle GMM avec k clusters
    gmm = GaussianMixture(n_components=k, covariance_type="spherical",init_params='k-means++', n_init=10)
    
    # Entraînement du modèle et calcul du BIC
    gmm.fit(load_pca)
    bic.append(gmm.bic(load_pca))
    
    # Prédiction des clusters et calcul de la silhouette
    clusters_gmm = gmm.fit_predict(load_pca)
    silhouette.append(silhouette_score(load_pca, clusters_gmm, metric='euclidean'))

# Affichage des graphiques avec subplots
plt.figure(figsize=(12, 6))

# Graphique pour la sélection du nombre de clusters avec BIC
plt.subplot(1, 2, 1)
plt.plot(range(2, k_max), bic, marker='o', linestyle='-')
plt.title("Sélection du nombre de clusters avec BIC")
plt.xlabel("Nombre de clusters")
plt.ylabel("BIC")

# Graphique pour la sélection du nombre de clusters avec silhouette
plt.subplot(1, 2, 2)
plt.plot(range(2, k_max), silhouette, marker='o', linestyle='-')
plt.title("Sélection du nombre de clusters avec silhouette")
plt.xlabel("Nombre de clusters")
plt.ylabel("Score de silhouette")

# Affichage global
plt.tight_layout()
plt.show()

On a essayé dans cette partie à chercher le nombre de cluster optimale pour GMM en utlisant deux critères différents : BIC et Silhouette. Pour le critère Silhouette, on cherche un intervalle ou l'inertie intra-classe est stable et on choisit le plus petit nombre de clusters correspondant à cet intervalle.

On fixe ici n = 5 clusters et on l'applique sur les données traitées par l'ACP afin de pouvoir afficher les résultats en fonction des 2 premières composantes principales. Comme l'estimation de la vraisemblance nécessite les paramètres de l'itération précédente, l'algorithme doit initialiser un modèle. Ici, cela est fait par défaut en utilisant KMeans++.

In [ ]:
# méthode GMM sur les données pca
n_components = 5
gmm = GaussianMixture(n_components = n_components, covariance_type="spherical",init_params='k-means++', n_init=10).fit(load_pca)

Les composantes du mélange gaussien initiales seront initialisées en utilisant l'algorithme k-means++, ce qui signifie que les centres initiaux seront choisis de manière à maximiser la distance entre eux. l'algorithme est exécuté 10 fois avec différentes initialisations aléatoires. La meilleure solution finale sera sélectionnée parmi ces exécutions. Cela peut aider à éviter les problèmes de convergence vers des optimas locaux et à améliorer la qualité de la solution finale.

In [ ]:
# identification des classes
classesGMM = gmm.predict(load_pca)

# Effectifs des classes
class_counts = pd.DataFrame(classesGMM).value_counts()

# Plot des effectifs des classes
plt.figure(figsize=(10, 6))
class_counts.plot(kind='bar')
plt.xlabel('Classes')
plt.ylabel('Effectifs')
plt.xticks(rotation=0)
plt.title('Histogramme des effectifs des classes')
plt.grid(True)
plt.show()

On voit que avec GMM, la répartion des individus entre les clusters est homogéne pour K = 5 clusters.

In [ ]:
# Visualisation des clusters
sns.scatterplot(x = load_pca[:, 0], y = load_pca[:, 1], hue=classesGMM, palette='viridis', legend='full')
plt.title('Visualisation des clusters avec GMM')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.show()

Ce graphique nous montre la distribution des stations dans 5 clusters obtenus à l'aide de la méthode GMM, dans le plan formé par les deux premières composantes principales de l'ACP. Nous retrouvons bien les commentaires effectués précédemment, puisque nous voyons un chevauchement de points assez conséquent entre certains clusters.

Les résultats avec GMM sont plutôt satisfaisants étant donné que seulement certains points sont mélangés entre les clusters. 

#####Cela peut être due au fait que l'hypothése "les distributions dans le data frame Load_pca sont guaussiennes" n'est pas validée.

In [ ]:
h = 30

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map',
                        color = classesGMM, color_continuous_scale = px.colors.sequential.Plasma_r, 
                        zoom  = 10, opacity = 1,
                        title = 'Clustering des stations ')

fig.show()

On affiche les résultats de GMM sur une carte intéractive à l'aide de la librairie plotly.express

Cette carte intéractive nous permet d'apprécier la répartitions des stations des différents clusters. Nous observons que les clusters sont séparés, ils ne forment pas 5 blocs compacts. On remarque aussi que GMM a fait une distinction entre les stations en altitude en les séparant majoritairement dans 2 clusters.

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic


table = pd.crosstab(coord["bonus"], classesGMM)

# Affichez la table de contingence
print(table)

# Tracez un mosaic plot 
plt.figure(figsize=(10, 6))
mosaic(table.stack(), title='Mosaic Plot entre les classes et la variable Bonus pour les collines')
plt.show()

On voit sur le mosaic plot que les stations en altitude ont majoritairement été placées dans 2 clusters différents. Mais ces 2 clusters ne sont pas entièrement composé de stations en altitude.

### Confirmation du choix de la dimension 

In [ ]:
p = 45  # nb d'individues
n= 4  #nb de groupes
plt.figure(figsize=(12, 8))

# Boucle sur les clusters
for cluster in range(5):
    
    indices_cluster = np.where(classesGMM == cluster)[0]
    
    for i in range(4):
        # Sélection de p indices aléatoires
        np.random.shuffle(indices_cluster)
        indices_to_plot = indices_cluster[:p]
        
        # Calcul de la moyenne des chargements pour ces individus
        mean_loading = np.mean(loading.iloc[indices_to_plot], axis=0)
        
        # Tracé de la moyenne des chargements
        plt.plot(mean_loading, label=f"Cluster {cluster}, Group {i+1}", marker='o')


    # Ajouter des légendes et des étiquettes
    plt.xlabel("Composantes principales")
    plt.ylabel("Loading moyen")
    plt.title("Loading moyen pour classe " + str(cluster))
    plt.legend()
    plt.grid(True)
    plt.show()

On constate de manière similaire à KMeans des comportements homogènes au sein des clusters. Cependant on notera que l'un des clusters contenant les stations en altitudes a un comportement plus erratique. On peut peut-être l'expliquer par le fait que beaucoup de stations ont un chargement moyen assez faible au cours de la semaine sans pour antant partager la même tendance.

### Comaparaison GMM et KMeans

In [ ]:
from scipy.spatial.distance import cdist
from matplotlib.patches import Ellipse

def plotKmeans(kmeans, data, n_clusters):
    kmeans.fit(data)
    clusters_kmeans = kmeans.predict(data)

    ax = plt.gca()
    ax.axis('equal')
    cmap = plt.get_cmap('Set3', n_clusters)

    # plot the input data
    ax.scatter(data[:, 0], data[:, 1], c=clusters_kmeans, s=1, linewidths=1, cmap=cmap)
    
    # plot the representation of the KMeans model
    centers = kmeans.cluster_centers_
    radius = [cdist(data[clusters_kmeans == i], [center]).max() for i, center in enumerate(centers)]
    for i in range(n_clusters):
        ax.add_patch(plt.Circle(centers[i], radius[i], fc=cmap.colors[i], alpha=0.3))


def draw_ellipse(mean, covariance, alpha, ax, col='#CCCCCC'):
    """Draw an ellipse with a given position and covariance"""    
    # Convert covariance to principal axes
    U, s, Vt = np.linalg.svd(covariance)
    angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
    width, height = np.sqrt(s)

    # Draw the Ellipse
    ax.add_patch(Ellipse(mean, 2*width,2* height, angle=angle, alpha=alpha, fc=col))

def plotGMM(gmm, data, n_clusters):
    gmm.fit(data)
    clusters_gmm = gmm.predict(data)
    
    ax = plt.gca()
    ax.axis('equal')
    cmap = plt.get_cmap('Set3', n_clusters)
    
    # plot the input data
    ax.scatter(data[:, 0], data[:, 1], c=clusters_gmm, s=1, linewidths=1, cmap=cmap)
    print(gmm.covariances_)
    # w_factor = 0.2 / gmm.weights_.max()
    for i in range(n_clusters):
        mean = gmm.means_[i,:2]
        covariance = gmm.covariances_[i,:,:2]
        w = gmm.weights_[i]
        draw_ellipse(mean, covariance, w, ax, cmap.colors[i])

def draw_ellipse_sph(mean, variance, alpha, ax, col='#CCCCCC'):
    """Draw an ellipse with a given position and variance"""    
    # Compute width and height from variance
    width= height = 6 * np.sqrt(variance)

    # Draw the Ellipse
    ax.add_patch(Ellipse(mean, width, height, alpha=alpha, color=col))

def plotGMM_sph(gmm, data, n_clusters):
    gmm.fit(data)
    clusters_gmm = gmm.predict(data)
    
    ax = plt.gca()
    ax.axis('equal')
    cmap = plt.get_cmap('Set3', n_clusters)
    
    # plot the input data
    ax.scatter(data[:, 0], data[:, 1], c=clusters_gmm, s=1, linewidths=1, cmap=cmap)
    
    for i in range(n_clusters):
        mean = gmm.means_[i,:2]
        variance = gmm.covariances_[i]
        w = gmm.weights_[i]
        draw_ellipse_sph(mean, variance, w, ax, cmap.colors[i])


In [ ]:
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters,init='k-means++', n_init='auto' )

plotKmeans(kmeans, load_pca,n_clusters)

On sait que Kmeans fait l'hypothèse que les clusters sont sphériques, donc dans notre cas, on visualise ces clusters avec la fonction plotKmeans qui crée un cercle pour chaque cluster dont le centre est le centroide corresopndant et de rayon qui est égale à la plus grande distance entre le centroide et un point de ce cluster. Cette représentation nous permet de constater que cette hypothèses rend Kmeans très sensible aux outliers.

In [ ]:
n_components = 5
gmm = GaussianMixture(n_components = n_components,covariance_type="spherical",init_params='k-means++', n_init=10)

plotGMM_sph(gmm, load_pca, n_components)

# Comparaison de 3 méthodes

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
n_clusters = 5


# Gaussian Mixture Model (GMM)
gmm = GaussianMixture(n_components=n_clusters, covariance_type="spherical", n_init=10, init_params='k-means++')
clusters_gmm = gmm.fit_predict(load_pca)
sns.scatterplot(x=load_pca[:, 0], y=load_pca[:, 1], hue=clusters_gmm, palette='viridis', legend='full', ax=axs[0])
axs[0].set_title('GMM Clustering')
axs[0].set_xlabel('Composante Principale 1 (PC1)')
axs[0].set_ylabel('Composante Principale 2 (PC2)')

# KMeans
kmeans = KMeans(n_clusters=n_clusters)
clusters_kmeans = kmeans.fit_predict(load_pca)
sns.scatterplot(x=load_pca[:, 0], y=load_pca[:, 1], hue=clusters_kmeans, palette='viridis', legend='full', ax=axs[1])
axs[1].set_title('KMeans Clustering')
axs[1].set_xlabel('Composante Principale 1 (PC1)')
axs[1].set_ylabel('Composante Principale 2 (PC2)')

# CAH avec liaison Ward
cah_ward = AgglomerativeClustering(n_clusters=4, linkage='ward')
clusters_cah_ward = cah_ward.fit_predict(load_pca)
sns.scatterplot(x=load_pca[:, 0], y=load_pca[:, 1], hue=clusters_cah_ward, palette='viridis', legend='full', ax=axs[2])
axs[2].set_title('CAH avec Liaison Ward')
axs[2].set_xlabel('Composante Principale 1 (PC1)')
axs[2].set_ylabel('Composante Principale 2 (PC2)')

plt.tight_layout()
plt.show()

A présent, nous allons comparer les 3 méthodes de clustering que nous avons étudiés. Tout d'abord, nous pouvons voir que la méthode de CAH est la moins efficace pour la séparation des clusters. En effet, nous voyons qu'il y a un chevauchement important des points entre les clusters, et que les frontières des différents clusters ne sont pas clairement identifiables contrairement aux méthodes K-means et Gaussian Mixture Model. Ensuite, nous pouvons voir que k means et GMM sont très efficaces pour la séparation des clusters. Cependant, on observe que K means permet une séparation plus prononcée des clusters, cela peut s'expliquer par le fait que k means est une méthode que l'on pourrait qualifier de hard clustering. En effet, pour la méthode k means les points sont "forcés" à appartenir à un cluster. A l'inverse pour la méthode GMM, que l'on pourrait qualifier de soft clustering, les points possèdent une probabilité d'appartenance à chacun des clusters.  En outre, on a vue que avec GMM nous arrivons à rentrer un peu plus dans les données qu'avec les autres méthodes vu que GMM arrive à faire une distinction entre les stations sur les collines et donc à identifier deux comportements différents entre ces stations.
on peut aussi expliquer les légers chevauchements de points de la méthode GMM, par le fait que " la représentation n'est pas optimale".

A présent, nous allons comparer sur la carte de Paris les 3 méthodes de clustering que nous avons étudiés.

In [ ]:
h = 30 

hours = np.arange(h, 168, 24)
cluster = clusters_pca

# --- # 

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map',
                        color = cluster, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Clustering KMeans des stations avec données traités par l\'ACP ')

fig.show()


In [ ]:
h = 30 

hours = np.arange(h, 168, 24)
cluster = clusters_pca

# --- # 

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map',
                        color = clusters_gmm, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Clustering GMM des stations avec données traités par l\'ACP ')

fig.show()


In [ ]:

h = 30 

hours = np.arange(h, 168, 24)
cluster = clusters_pca

# --- # 

fig = px.scatter_mapbox(coord, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map',
                        color = clusters_cah_ward, color_continuous_scale = px.colors.sequential.Plasma_r, #size = load_per_hour,
                        zoom  = 10, opacity = .9,
                        title = 'Clustering CAH des stations avec données traités par l\'ACP ')

fig.show()

# CONCLUSION

Nous avons atteint l'objectif initial qui était d'identifier des clusters significatifs dans nos données. Cependant nous avons observé qu'il était difficile d'assurer une homogénéité parfaite au sein des clusters. De plus, le format assez particulier de nos données nous a empêché d'avoir des clusters toujours bien séparés. En outre, nous avons vu que les méthodes de clustering dépendaient du contexte (ici Kmeans est très efficace contrairement à CAH) et qu'il n'est pas conseillé de suivre aveuglément les résultats de nos méthodes sans les remettre en question (cf. choix du nombre idéal de clusters).ers)